# Base-wise conservation of introns


First, load the memory-mapped pickle file that was created in notebook [0.2.09]

In [ ]:
%%time

import cPickle as pickle

memmap_dir = '/home/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap'
filename = '{}/hg19_phastcons_placental_mammal_htseq.pickle'.format(memmap_dir)
! ls -lh $filename
with open(filename) as f:
    conservation = pickle.load(f)

-rw-r--r-- 1 obotvinnik yeo-group 79G Sep 26 07:33 /home/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap/hg19_phastcons_placental_mammal_htseq.pickle


In [4]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/'

alt_exons_bedfile = '{}/exon2.bed'.format(folder)
# constitutive_bedfile = '{}/constitutive_exons.bed'.format(folder)

In [5]:
%%time
import glob
import os
import numpy as np
import pandas as pd
import pybedtools
import HTSeq
from flotilla.util import timestamp

bedfiles = (alt_exons_bedfile,) #constitutive_bedfile

directions = 'upstream', 'downstream'

nt = 200


bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

for bedfile in bedfiles:
    bed = pybedtools.BedTool(bedfile)

    basename = os.path.basename(bedfile)
    prefix = basename.split('.bed')[0]
    
    for direction in directions:
        if direction == 'downstream':
            # Get downstream intron
            intron = bed.flank(l=0, r=nt, s=True, g=pybedtools.chromsizes('hg19'))
        else:
            intron = bed.flank(l=nt, r=0, s=True, g=pybedtools.chromsizes('hg19'))
        
        # Get just unique upstream,/downstream
        intron = pybedtools.BedTool(list(set(x for x in intron)))
        nrow = len(intron)
        ncol = nt
        array = np.zeros(shape=(nrow, ncol), dtype=float)
        junction_ids = pd.Series(index=np.arange(nrow))

        print 'Iterating over {} intervals in {} {}nt of {} ...'.format(nrow, direction, nt, basename)
        for i, interval in enumerate(intron):
            if (i+1) % 10000 == 0:
                print '\t{}\t{}/{}'.format(timestamp(), i+1, nrow)
            junction_ids[i] = interval.name
            region = conservation[HTSeq.GenomicInterval(interval.chrom, interval.start, interval.stop)]
            count = sum(1 for _ in region.values())
            subset = np.fromiter(region.values(),
                                 dtype=float, count=count)
            if interval.strand == '-':
                subset = subset[::-1]
                j = nt - count
                array[i][j:] = subset
            else:
                j = count
                array[i][:j] = subset
        intron_conservation = pd.DataFrame(array, index=junction_ids.values)
        filename = '{}/{}_{}{}_placental_mammal_conservation.csv'.format(bed_folder, prefix, direction, nt)
        print '\t', filename
        intron_conservation.to_csv(filename)

Iterating over 33693 intervals in upstream 200nt of exon2.bed ...
	2015-10-30 10:30:14	10000/33693
	2015-10-30 10:30:16	20000/33693
	2015-10-30 10:30:17	30000/33693
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2_upstream200_placental_mammal_conservation.csv
Iterating over 33693 intervals in downstream 200nt of exon2.bed ...
	2015-10-30 10:30:25	10000/33693
	2015-10-30 10:30:27	20000/33693
	2015-10-30 10:30:28	30000/33693
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2_downstream200_placental_mammal_conservation.csv
CPU times: user 21.1 s, sys: 326 ms, total: 21.4 s
Wall time: 22.3 s
